In [100]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [101]:
pd.set_option('display.max_colwidth', 100)  # Limit column width for better readability
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns


In [102]:
# Load training and testing data
train_data = pd.read_csv(r'C:\Users\padhee.3\Downloads\Take Home Project\training_processed_data.csv')  # Replace with your training file path
inference_data = pd.read_csv(r'C:\Users\padhee.3\Downloads\Take Home Project\testing_processed_data.csv')    # Replace with your testing file path

In [103]:
print(train_data.columns)
print(inference_data.columns)

Index(['id', 'loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership',
       'annual_inc', 'purpose', 'percent_bc_gt_75', 'bc_util', 'dti',
       'inq_last_6mths', 'mths_since_recent_inq', 'revol_util',
       'total_bc_limit', 'tot_cur_bal', 'bad_flag'],
      dtype='object')
Index(['id', 'loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership',
       'annual_inc', 'purpose', 'percent_bc_gt_75', 'bc_util', 'dti',
       'inq_last_6mths', 'mths_since_recent_inq', 'revol_util',
       'total_bc_limit', 'tot_cur_bal', 'bad_flag'],
      dtype='object')


In [104]:
# Drop the 'id' column 
if 'id' in train_data.columns:
    train_data = train_data.drop(columns=['id'])


In [105]:
train_data.head(2)

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,percent_bc_gt_75,bc_util,dti,inq_last_6mths,mths_since_recent_inq,revol_util,total_bc_limit,tot_cur_bal,bad_flag
0,7550,36 months,0.1624,3.0,RENT,28000.0,debt_consolidation,100.0,96.0,8.40,0.0,17.0,0.720,4000.0,5759.0,0.0
1,27050,36 months,0.1099,10.0,OWN,55000.0,debt_consolidation,25.0,53.9,22.87,0.0,8.0,0.612,35700.0,114834.0,0.0


In [106]:
print("Train dataset shape:", train_data.shape)


Train dataset shape: (189457, 16)


In [107]:
# Check the distribution of the target variable
class_counts = train_data['bad_flag'].value_counts()
print("Class Distribution:")
print(class_counts)

# Calculate imbalance ratio
imbalance_ratio = class_counts.min() / class_counts.max()
print(f"Imbalance Ratio: {imbalance_ratio:.2f}")

Class Distribution:
bad_flag
0.0    176329
1.0     13128
Name: count, dtype: int64
Imbalance Ratio: 0.07


###An imbalance ratio of 0.07 indicates a highly imbalanced dataset, with the majority class being much more frequent than the minority class. This will likely cause the model to be biased towards predicting the majority class, resulting in poor performance for the minority class.

#We can try oversampling, undersampling, or weighted loss.

In [108]:
# Define categorical and numerical columns
categorical_columns = ['home_ownership', 'purpose', 'term']
target_column = 'bad_flag'
numerical_columns = [col for col in df_train_balanced.columns if col not in categorical_columns + [target_column]]

In [109]:
# Separate majority and minority classes
df_majority = train_data[train_data[target_column] == 0]
df_minority = train_data[train_data[target_column] == 1]

# Undersample the majority class using stratified sampling
df_majority_undersampled = resample(
    df_majority,
    replace=False,              # Sample without replacement
    n_samples=len(df_minority), # Match minority class size
    random_state=42,            # For reproducibility
    stratify=df_majority[target_column]  # Stratify to preserve class distribution
)

# Combine undersampled majority class with minority class
df_train_balanced = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the dataset
df_train_balanced = df_train_balanced.sample(frac=1, random_state=42)

# Verify the new distribution
print("Balanced Class Distribution:")
print(df_train_balanced[target_column].value_counts())

Balanced Class Distribution:
bad_flag
1.0    13128
0.0    13128
Name: count, dtype: int64


In [110]:
# Define preprocessing pipeline for categorical and numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),  # Scale numerical columns
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)  # One-hot encode categorical columns
    ]
)

# Separate features (X) and target (y) in the balanced dataset
X_balanced = df_train_balanced.drop(columns=[target_column])
y_balanced = df_train_balanced[target_column].values

# Apply preprocessing to the balanced dataset
X_balanced_transformed = preprocessor.fit_transform(X_balanced)

# Convert the transformed data into tensors
X_train_tensor = torch.tensor(X_balanced_transformed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_balanced, dtype=torch.float32).unsqueeze(1)

# Verify the new shapes
print("X_train_tensor shape:", X_train_tensor.shape)
print("y_train_tensor shape:", y_train_tensor.shape)

X_train_tensor shape: torch.Size([26256, 32])
y_train_tensor shape: torch.Size([26256, 1])


In [111]:
# Inspect the number of unique categories in each categorical column
for col in categorical_columns:
    print(f"{col}: {df_train_balanced[col].nunique()} unique categories")

# Check the shape of the transformed feature matrix
print("Transformed X_balanced shape:", X_balanced_transformed.shape)

home_ownership: 5 unique categories
purpose: 13 unique categories
term: 2 unique categories
Transformed X_balanced shape: (26256, 32)


In [112]:
# Print unique categories
categories = df_train_balanced['home_ownership'].unique()
print("Categories in 'home_ownership':", categories)

categories = df_train_balanced['purpose'].unique()
print("Categories in 'purpose':", categories)

categories = df_train_balanced['term'].unique()
print("Categories in 'term':", categories)

categories = test_data['home_ownership'].unique()
print("Categories in 'home_ownership':", categories)

categories = test_data['purpose'].unique()
print("Categories in 'purpose':", categories)

categories = test_data['term'].unique()
print("Categories in 'term':", categories)

Categories in 'home_ownership': ['MORTGAGE' 'RENT' 'OWN' 'OTHER' 'NONE']
Categories in 'purpose': ['debt_consolidation' 'other' 'medical' 'credit_card' 'car' 'wedding'
 'renewable_energy' 'vacation' 'home_improvement' 'major_purchase'
 'small_business' 'moving' 'house']
Categories in 'term': [' 36 months' ' 60 months']
Categories in 'home_ownership': ['RENT' 'OWN' 'MORTGAGE' 'NONE' 'OTHER']
Categories in 'purpose': ['debt_consolidation' 'home_improvement' 'credit_card' 'other'
 'major_purchase' 'small_business' 'house' 'moving' 'medical' 'car'
 'vacation' 'renewable_energy' 'wedding']
Categories in 'term': [' 36 months' ' 60 months']


###There is a mismatch in categories in purpose as inference data has an unseen category of renewable energy. Hence, for now, I will fit the encoding only on training data and address unseen categories to be ignored.

In [113]:
from sklearn.model_selection import train_test_split

# Split the data into 80% train and 20% validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_tensor, y_train_tensor, test_size=0.2, random_state=42
)

# Verify the new shapes after splitting
print("X_train_split shape:", X_train_split.shape)
print("y_train_split shape:", y_train_split.shape)
print("X_val_split shape:", X_val_split.shape)
print("y_val_split shape:", y_val_split.shape)


X_train_split shape: torch.Size([21004, 32])
y_train_split shape: torch.Size([21004, 1])
X_val_split shape: torch.Size([5252, 32])
y_val_split shape: torch.Size([5252, 1])


In [114]:
# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(X_train_split, y_train_split)
val_dataset = TensorDataset(X_val_split, y_val_split)

# Create DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Verify the DataLoader setup
print(f"Train loader: {len(train_loader)} batches")
print(f"Validation loader: {len(val_loader)} batches")

Train loader: 657 batches
Validation loader: 165 batches


In [87]:
# Define the neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(32, 64)  # Input size 32, output size 64
        self.fc2 = nn.Linear(64, 32)  # Input size 64, output size 32
        self.fc3 = nn.Linear(32, 1)   # Output layer (binary classification)
        
        # Adding Dropout layers
        self.dropout = nn.Dropout(0.3)  # Dropout rate of 30%
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout after the first layer
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout after the second layer
        x = torch.sigmoid(self.fc3(x))  # Sigmoid for binary classification
        return x

# Initialize the model
input_size = X_train.shape[1]
hidden_size = 64  # Configurable number of neurons in the hidden layer
model = NeuralNetwork(input_size, hidden_size)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  

In [88]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

Epoch 1/20, Train Loss: 0.6408, Validation Loss: 0.6286
Epoch 2/20, Train Loss: 0.6321, Validation Loss: 0.6247
Epoch 3/20, Train Loss: 0.6280, Validation Loss: 0.6215
Epoch 4/20, Train Loss: 0.6257, Validation Loss: 0.6196
Epoch 5/20, Train Loss: 0.6239, Validation Loss: 0.6161
Epoch 6/20, Train Loss: 0.6235, Validation Loss: 0.6152
Epoch 7/20, Train Loss: 0.6223, Validation Loss: 0.6143
Epoch 8/20, Train Loss: 0.6216, Validation Loss: 0.6138
Epoch 9/20, Train Loss: 0.6209, Validation Loss: 0.6122
Epoch 10/20, Train Loss: 0.6201, Validation Loss: 0.6121
Epoch 11/20, Train Loss: 0.6199, Validation Loss: 0.6118
Epoch 12/20, Train Loss: 0.6191, Validation Loss: 0.6100
Epoch 13/20, Train Loss: 0.6183, Validation Loss: 0.6106
Epoch 14/20, Train Loss: 0.6184, Validation Loss: 0.6092
Epoch 15/20, Train Loss: 0.6176, Validation Loss: 0.6095
Epoch 16/20, Train Loss: 0.6174, Validation Loss: 0.6090
Epoch 17/20, Train Loss: 0.6166, Validation Loss: 0.6097
Epoch 18/20, Train Loss: 0.6174, Validat

The validation loss is still higher than the training loss, which might indicate that the model is overfitting to the training data.
The loss is still high with learning rate varying 0.001 - 0.005. Next, adding drop out/regularization, add grid search for hyperparameter tuning.

In [89]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Add performance evaluation after training
def evaluate_model(model, val_loader, y_val_tensor):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            preds = (outputs >= 0.5).int()  # Binary classification
            all_preds.append(preds)
            all_labels.append(y_batch)
    
    # Flatten the lists
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_preds)

    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation Precision: {precision:.4f}")
    print(f"Validation Recall: {recall:.4f}")
    print(f"Validation F1-Score: {f1:.4f}")
    print(f"Validation AUC: {auc:.4f}")

# After training, evaluate the model on the validation set
evaluate_model(model, val_loader, y_val_tensor)

Validation Accuracy: 0.6645
Validation Precision: 0.6501
Validation Recall: 0.7144
Validation F1-Score: 0.6807
Validation AUC: 0.6644


In [127]:
#Added two hidden layers (hidden_size1 and hidden_size2) for increased model complexity. 
#LeakyReLU Activation: Replaced ReLU with LeakyReLU for better gradient flow, especially for negative inputs. 
#Applied dropout after each hidden layer to reduce overfitting. 
# Define the input and output sizes
input_size = X_train_tensor.shape[1]  # Number of features (columns) in the input data
hidden_size1 = 64  # First hidden layer size
hidden_size2 = 32   # Second hidden layer size
output_size = 1     # For binary classification, using 1.

class NeuralNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(NeuralNetwork, self).__init__()
        # Layer definitions
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.act1 = nn.LeakyReLU(negative_slope=0.01)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.act2 = nn.LeakyReLU(negative_slope=0.01)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_size2, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.act2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        return x

In [132]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

# Training function
def train(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)
    return avg_train_loss

# Validation function
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    return avg_val_loss

# Hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.01],
    'dropout_rate': [0.4, 0.5],
    'weight_decay': [1e-5, 1e-4],
    'batch_size': [32, 64]  # Added batch size for experimentation
}

# Create a function that returns a model with the given hyperparameters
def create_model(learning_rate, dropout_rate, weight_decay):
    model = NeuralNetwork(dropout_rate=dropout_rate)
    optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    return model, optimizer

# Dataset loading
train_data = TensorDataset(X_train_tensor, y_train_tensor)  
val_data = TensorDataset(X_val_tensor, y_val_tensor)

# Hyperparameter tuning with Early Stopping
best_val_loss = float('inf')
best_params = None
patience = 10  # Early stopping patience
num_epochs = 50

for learning_rate in param_grid['learning_rate']:
    for dropout_rate in param_grid['dropout_rate']:
        for weight_decay in param_grid['weight_decay']:
            for batch_size in param_grid['batch_size']:
                print(f"Training with LR: {learning_rate}, Dropout: {dropout_rate}, Weight Decay: {weight_decay}, Batch Size: {batch_size}")
                
                # Create DataLoader with current batch size
                train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
                
                # Create model and optimizer with the current hyperparameters
                model, optimizer = create_model(learning_rate, dropout_rate, weight_decay)
                
                # Initialize loss function and scheduler
                criterion = nn.BCEWithLogitsLoss()
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
                
                # Early Stopping variables
                patience_counter = 0
                best_model_loss = float('inf')

                # Train the model for a certain number of epochs
                for epoch in range(num_epochs):
                    train_loss = train(model, train_loader, optimizer, criterion)
                    val_loss = validate(model, val_loader, criterion)
                    
                    # Step the scheduler after each epoch
                    scheduler.step(val_loss)
                    
                    # Print losses for each epoch
                    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
                    
                    # Early Stopping logic
                    if val_loss < best_model_loss:
                        best_model_loss = val_loss
                        patience_counter = 0
                        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
                    else:
                        patience_counter += 1
                    
                    if patience_counter >= patience:
                        print(f"Early stopping at epoch {epoch+1}")
                        break

                # Save the best hyperparameters
                if best_model_loss < best_val_loss:
                    best_val_loss = best_model_loss
                    best_params = {
                        'learning_rate': learning_rate,
                        'dropout_rate': dropout_rate,
                        'weight_decay': weight_decay,
                        'batch_size': batch_size
                    }

                print(f"Best parameters so far: {best_params}")

# After all iterations, print the best parameters
print("Best hyperparameters found:", best_params)


Training with LR: 0.001, Dropout: 0.4, Weight Decay: 1e-05, Batch Size: 32
Epoch 1/50, Train Loss: 0.6624, Validation Loss: 0.6649
Epoch 2/50, Train Loss: 0.6514, Validation Loss: 0.6805
Epoch 3/50, Train Loss: 0.6475, Validation Loss: 0.7206
Epoch 4/50, Train Loss: 0.6445, Validation Loss: 0.7102
Epoch 5/50, Train Loss: 0.6426, Validation Loss: 0.7169
Epoch 6/50, Train Loss: 0.6387, Validation Loss: 0.7260
Epoch 7/50, Train Loss: 0.6388, Validation Loss: 0.7261
Epoch 8/50, Train Loss: 0.6394, Validation Loss: 0.7294
Epoch 9/50, Train Loss: 0.6378, Validation Loss: 0.7336
Epoch 10/50, Train Loss: 0.6371, Validation Loss: 0.7334
Epoch 11/50, Train Loss: 0.6377, Validation Loss: 0.7335
Early stopping at epoch 11
Best parameters so far: {'learning_rate': 0.001, 'dropout_rate': 0.4, 'weight_decay': 1e-05, 'batch_size': 32}
Training with LR: 0.001, Dropout: 0.4, Weight Decay: 1e-05, Batch Size: 64
Epoch 1/50, Train Loss: 0.6643, Validation Loss: 0.6614
Epoch 2/50, Train Loss: 0.6523, Valida

In [133]:

# Load the best model
best_model = NeuralNetwork(dropout_rate=best_params['dropout_rate'])
best_model.load_state_dict(torch.load('best_model.pth'))  # Load saved weights

# Evaluate on the validation dataset
evaluate_model(best_model, val_loader, y_val_tensor)



Validation Accuracy: 0.5392
Validation Precision: 0.6748
Validation Recall: 0.1568
Validation F1-Score: 0.2545
Validation AUC: 0.5404


C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\3037868007.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_model.pth')) 

In [65]:
print("Balanced dataset shape:", df_train_balanced.shape)


Balanced dataset shape: (26256, 16)


In [66]:
print("Columns in balanced dataset:", df_train_balanced.columns)


Columns in balanced dataset: Index(['loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership',
       'annual_inc', 'purpose', 'percent_bc_gt_75', 'bc_util', 'dti',
       'inq_last_6mths', 'mths_since_recent_inq', 'revol_util',
       'total_bc_limit', 'tot_cur_bal', 'bad_flag'],
      dtype='object')


In [70]:
# Define model, loss function, optimizer, and learning rate scheduler
model = NeuralNetwork(input_size, hidden_size)

criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Implement learning rate scheduler (for example, reduce learning rate on plateau)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)

# Loop over each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_tensor)):
    print(f"Training fold {fold + 1}...")

    # Split the data into train and validation sets
    X_train_fold, X_val_fold = X_train_tensor[train_idx], X_train_tensor[val_idx]
    y_train_fold, y_val_fold = y_train_tensor[train_idx], y_train_tensor[val_idx]

    # Create DataLoader for the current fold
    train_dataset = TensorDataset(X_train_fold, y_train_fold)
    val_dataset = TensorDataset(X_val_fold, y_val_fold)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(20):  # Adjust number of epochs as needed
        model.train()
        for batch in train_loader:
            X_batch, y_batch = batch
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # Step the scheduler after each epoch
        scheduler.step(loss)  # The scheduler will reduce the learning rate if validation loss plateaus

    # Validate the model (same as before)
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch in val_loader:
            X_batch, y_batch = batch
            outputs = model(X_batch)
            y_true.extend(y_batch.numpy())
            y_pred.extend(torch.sigmoid(outputs).numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate performance metrics for the current fold
    accuracy = accuracy_score(y_true, y_pred > 0.5)
    precision = precision_score(y_true, y_pred > 0.5)
    recall = recall_score(y_true, y_pred > 0.5)
    f1 = f1_score(y_true, y_pred > 0.5)
    auc = roc_auc_score(y_true, y_pred)

    # Append metrics to lists
    fold_accuracies.append(accuracy)
    fold_precisions.append(precision)
    fold_recalls.append(recall)
    fold_f1_scores.append(f1)
    fold_aucs.append(auc)

    print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

# Calculate average metrics across all folds
avg_accuracy = np.mean(fold_accuracies)
avg_precision = np.mean(fold_precisions)
avg_recall = np.mean(fold_recalls)
avg_f1 = np.mean(fold_f1_scores)
avg_auc = np.mean(fold_aucs)

print(f"\nAverage Metrics Across All Folds:")
print(f"Accuracy: {avg_accuracy:.4f}")
print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"F1-Score: {avg_f1:.4f}")
print(f"AUC: {avg_auc:.4f}")


C:\Users\padhee.3\AppData\Local\anaconda3\envs\myenv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training fold 1...
Fold 1 - Accuracy: 0.5029, Precision: 0.5022, Recall: 1.0000, F1: 0.6686, AUC: 0.6801
Training fold 2...
Fold 2 - Accuracy: 0.4997, Precision: 0.4993, Recall: 0.9996, F1: 0.6660, AUC: 0.6919
Training fold 3...
Fold 3 - Accuracy: 0.4986, Precision: 0.4979, Recall: 0.9989, F1: 0.6645, AUC: 0.6952
Training fold 4...
Fold 4 - Accuracy: 0.5092, Precision: 0.5084, Recall: 1.0000, F1: 0.6741, AUC: 0.6965
Training fold 5...
Fold 5 - Accuracy: 0.4959, Precision: 0.4953, Recall: 1.0000, F1: 0.6625, AUC: 0.6789

Average Metrics Across All Folds:
Accuracy: 0.5013
Precision: 0.5006
Recall: 0.9997
F1-Score: 0.6671
AUC: 0.6885


#Model is performing well in terms of recall, achieving almost perfect recall across all folds (close to 1.0). However, the precision is relatively lower (around 0.5), and accuracy is also quite close to 0.5. This indicates that your model might be predicting a large number of false positives, which leads to a lower precision despite having high recall. Next, I will try oversampling as there is a huge class imbalance. 

In [71]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = train_data[train_data[target_column] == 0]
df_minority = train_data[train_data[target_column] == 1]

# Oversample the minority class
df_minority_oversampled = resample(
    df_minority,
    replace=True,               # Sample with replacement
    n_samples=len(df_majority), # Match majority class size
    random_state=42,            # For reproducibility
    stratify=df_minority[target_column]  # Stratify to preserve class distribution
)

# Combine oversampled minority class with majority class
df_train_balanced = pd.concat([df_majority, df_minority_oversampled])

# Shuffle the dataset
df_train_balanced = df_train_balanced.sample(frac=1, random_state=42)

# Verify the new distribution
print("Balanced Class Distribution:")
print(df_train_balanced[target_column].value_counts())


Balanced Class Distribution:
bad_flag
0.0    176329
1.0    176329
Name: count, dtype: int64


In [157]:
# Load and preprocess inference data
original_testing = pd.read_csv(r'C:\Users\padhee.3\Downloads\Take Home Project\testing_loan_data.csv')
print(original_testing.shape)


(102505, 23)


C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\1398523151.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  original_testing = pd.read_csv(r'C:\Users\padhee.3\Downloads\Take Home Project\testing_loan_data.csv')


In [158]:
columns_to_select = ['id', 'loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership',
                     'annual_inc', 'purpose', 'percent_bc_gt_75', 'bc_util', 'dti',
                     'inq_last_6mths', 'mths_since_recent_inq', 'revol_util',
                     'total_bc_limit', 'tot_cur_bal', 'bad_flag']

df_selected_columns = original_testing[columns_to_select]
print(df_selected_columns.shape)


(102505, 17)


In [160]:
# Clean 'years' column: Remove "years" and retain numbers
if 'emp_length' in df_selected_columns.columns:  # Replace 'years' with the actual column name
    df_selected_columns['emp_length'] = df_selected_columns['emp_length'].str.extract(r'(\d+)').astype(float)

# Clean percentage columns: Remove '%' and convert to decimal
percentage_columns = ['int_rate', 'revol_util']  # Replace with actual percentage column names
for col in percentage_columns:
    if col in df_selected_columns.columns:
        df_selected_columns[col] = df_selected_columns[col].str.replace('%', '', regex=True).astype(float) / 100

# Display cleaned dataset
print("Cleaned Columns:")
df_selected_columns.head(2)

Cleaned Columns:


C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\2005757757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['emp_length'] = df_selected_columns['emp_length'].str.extract(r'(\d+)').astype(float)
C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\2005757757.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns[col] = df_selected_columns[col].str.replace('%', '', regex=True).astype(float) / 100
C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\2005757757.py:9:

,id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,percent_bc_gt_75,bc_util,dti,inq_last_6mths,mths_since_recent_inq,revol_util,total_bc_limit,tot_cur_bal,bad_flag
0,20000001,10000,36 months,0.2215,8.0,RENT,37000.0,debt_consolidation,80.0,83.0,28.51,1,3.0,0.731,16200,36809,NaN
1,20000002,1400,36 months,0.1824,6.0,RENT,41000.0,other,0.0,0.0,26.58,0,9.0,0.115,4000,19536,NaN


In [161]:
df_selected_columns.isna().sum()


id                            0
loan_amnt                     0
term                          0
int_rate                      0
emp_length                 5321
home_ownership                0
annual_inc                    0
purpose                       0
percent_bc_gt_75           1046
bc_util                    1042
dti                           0
inq_last_6mths                0
mths_since_recent_inq      8828
revol_util                   48
total_bc_limit                0
tot_cur_bal                   0
bad_flag                 102505
dtype: int64

In [162]:
# Imputation strategy for each column
columns_to_impute = {
    'revol_util': 'median',  # Median for revolving utilization ratio
        'percent_bc_gt_75': 'median',  # Median for percentage, as it may be skewed
    'bc_util': 'median',  # Median for utilization ratio, which can be skewed
    
    'emp_length': 'mode',  # Mode for employment length (categorical)

    'mths_since_recent_inq': 'median'  # Median for months since recent inquiry
}

# Perform imputation based on the specified methods
for column, method in columns_to_impute.items():
    if method == 'mean':
        df_selected_columns[column] = df_selected_columns[column].fillna(df_selected_columns[column].mean())
    elif method == 'median':
        df_selected_columns[column] = df_selected_columns[column].fillna(df_selected_columns[column].median())
    elif method == 'mode':
        df_selected_columns[column] = df_selected_columns[column].fillna(df_selected_columns[column].mode()[0])

# Print missingness after imputation
missingness_after_imputation = df_selected_columns.isna().sum()
print("Missingness after imputation:")
print(missingness_after_imputation)

Missingness after imputation:
id                            0
loan_amnt                     0
term                          0
int_rate                      0
emp_length                    0
home_ownership                0
annual_inc                    0
purpose                       0
percent_bc_gt_75              0
bc_util                       0
dti                           0
inq_last_6mths                0
mths_since_recent_inq         0
revol_util                    0
total_bc_limit                0
tot_cur_bal                   0
bad_flag                 102505
dtype: int64


C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\2240313321.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns[column] = df_selected_columns[column].fillna(df_selected_columns[column].median())
C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\2240313321.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns[column] = df_selected_columns[column].fillna(df_selected_columns[column].mode()[0])


In [164]:
# Apply the same preprocessing to the inference data (excluding 'id' column for preprocessing)
X_inference_transformed = preprocessor.transform(df_selected_columns.drop(columns=['id']))

# Convert inference data to PyTorch tensor
X_inference_tensor = torch.tensor(X_inference_transformed, dtype=torch.float32)

# Inference
model.eval()
with torch.no_grad():
    predictions = model(X_inference_tensor).squeeze()
    predicted_labels = (predictions >= 0.5).int()

# Save predictions to the inference data
df_selected_columns['bad_flag'] = predicted_labels.numpy()

# Save results to a CSV file, including the 'id' column
df_selected_columns.to_csv(r'C:\Users\padhee.3\Downloads\Take Home Project\inference_results.csv', index=False)

print("Inference completed. Results saved to 'inference_results.csv'.")


C:\Users\padhee.3\AppData\Local\Temp\ipykernel_21620\905351146.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns['bad_flag'] = predicted_labels.numpy()


Inference completed. Results saved to 'inference_results.csv'.


In [ ]:
# Step 5: Add the predictions (bad_flag) to the original testing data using the 'id' column
original_testing['bad_flag'] = predicted_labels.numpy()

# Step 6: Save the updated testing data with predictions to a CSV file (including 'id')
original_testing.to_csv(r'C:\Users\padhee.3\Downloads\Take Home Project\testing_loan_data_results.csv', index=False)

print("Inference completed. Results saved to 'inference_results.csv'.")